# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU faiss_cpu pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

fatal: destination path 'DataRepository' already exists and is not an empty directory.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

159

In [10]:
documents[1]

Document(page_content='ELON MUSK, an individual, \nPlaintiff, \nvs. \nSAMUEL ALTMAN, an individual, GREGORY \nBROCKMAN, an individual, OPENAI, INC., a \ncorporation, OPENAI, L.P., a limited \npartnership, OPENAI, L.L.C., a limited liability \ncompany, OPENAI GP, L.L.C., a limited \nliability company, OPENAI OPCO, LLC, a \nlimited liability company, OPENAI GLOBAL, \nLLC, a limited liability company, OAI \nCORPORATION, LLC, a limited liability \ncompany, OPENAI HOLDINGS, LLC, a limited \nliability company, and DOES 1 through 100, \ninclusive, \nDefendants. \nCase No.:  \n[UNLIMITED JURISDICTION] \n \nCOMPLAINT FOR (1) BREACH OF \nCONTRACT, (2) PROMISSORY \nESTOPPEL, (3) BREACH OF FIDUCIARY \nDUTY, (4) UNFAIR COMPETITION', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 0, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modD

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [12]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

**Answer** FAISS is very flexible in terms of retrieval options. The default options seem to be Euclidean distance, dot product and cosine similarity (which is essentially a dot product on normalized vectors). However, FAISS also supports alternative retrieval options that offer various levels of compromise between memory, search speed and accuracy. Its GPU implementation significantly accelerates search operations, supporting native multi-GPU configurations. FAISS efficiently handles billions of vectors, crucial for large-scale datasets common in AI applications.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [13]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [14]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [15]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [16]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [17]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [18]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [19]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

**Answer**: Above we have a RAG chain that first uses Python's itemgetter to extract the "question" from input, passing it to a retriever but also keeping the original "question" intact. A RunnablePassthrough then temporarily holds the "context" (which is obtained as an output of the "question" chained into the retriever) without altering it. Finally, the "context" and "question" are used as inputs for a prompt for ChatOpenAI, generating a "response".

Let's test it out!

In [20]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [21]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [22]:
loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

eval_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

**Answer**:  Because we want to test whether the system can handle unseen data and diverse scenarios effectively, not just the specific conditions it was trained or optimized on. A different strategy might reveal strengths or weaknesses that were not apparent under the training conditions, providing a better understanding of the system's performance and areas for improvement.

In [23]:
len(eval_documents)

92

In [24]:
documents == eval_documents

False

In [25]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(eval_documents, test_size=12, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/188 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/12 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

**Answer** RAGAS provides a synthetic Q&A data genaration module that can cover different levels of complexity. First, 'simple' questions are generated where seeding is used to ensure diversity. Then, the original questions might undergo "evolutions", whereby they become more convolved. The new questions might require reasoning in order to be answered ('reasoning' questions), or might require information contained in multiple chunks ('multi_context' questions). This is a way to simulate the variability in queries that production RAG systems might receive. 

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question='What are some of the capabilities and achievements of the GPT-4 model compared to previous models like GPT-3?', contexts=['step manner, like a human, “by simply adding ‘Let’s think step by step’ before each answer.”  \n85. \nA path to Artificial Generative Intelligence could be seen. And the timeline to it was \ndramatically compressing. \n86. \nOn March 14, 2023, OpenAI released a new generation of its model, GPT-4. This \ngeneration was not just capable of reasoning but was better at reasoning than average humans. \nGPT-4 scored in the 90th percentile on the Uniform Bar Exam. It scored in the 99th percentile on \nthe GRE Verbal Assessment. It even scored a 77% on the Advanced Sommelier examination. By \nOpenAI’s own objective measures, GPT-4 is already capable of intelligence that is superior to \nhumans on a wide variety of economically valuable tasks. \n87. \nThis development was not lost on the research community. In a detailed analysis \ntitled “Sparks of Artifi

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [27]:
test_df = testset.to_pandas()

In [28]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What are some of the capabilities and achievem...,"[step manner, like a human, “by simply adding ...",GPT-4 is capable of solving novel and difficul...,simple,True
1,What is the purpose of the detailed paper rele...,[(GPT) along with a detailed paper describing ...,The purpose of the detailed paper released by ...,simple,True
2,How does the performance of the GPT-4 model co...,"[step manner, like a human, “by simply adding ...",The performance of the GPT-4 model is vastly s...,simple,True
3,What is Altman's role in the tech industry and...,[and uncontrolled growth have become religion ...,Altman has been one of the dedicated high prie...,reasoning,True
4,What model did OpenAI release in 2019 that had...,[(GPT) along with a detailed paper describing ...,OpenAI released GPT-2 in 2019 with a detailed ...,reasoning,True
5,Why did OpenAI release the full version of GPT...,[(GPT) along with a detailed paper describing ...,OpenAI released the full version of GPT-2 with...,reasoning,True
6,"""What is the role of GPT in deep learning and ...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,GPT is a large-language model developed by Ope...,multi_context,True
7,What did OpenAI's release of the GPT-2 model i...,[(GPT) along with a detailed paper describing ...,Their publication proved to be useful to the d...,multi_context,True
8,"""What advantages does the OpenAI business mode...","[new business model, they get the same “for pr...",The OpenAI business model offers the advantage...,multi_context,True
9,What are the defendants' fiduciary duties unde...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"Defendants owe fiduciary duties to Plaintiff, ...",multi_context,True


In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [32]:
response_dataset[0]

{'question': 'What are some of the capabilities and achievements of the GPT-4 model compared to previous models like GPT-3?',
 'answer': 'GPT-4 can solve novel and difficult tasks spanning mathematics, coding, vision, medicine, law, psychology, and more without needing special prompting. Its performance is strikingly close to human-level performance and often surpasses prior models like GPT-3.',
 'contexts': ['titled “Sparks of Artificial General Intelligence: Early experiments with GPT-4,” Microsoft \nresearchers note that “GPT-4 can solve novel and difficult tasks that span mathematics, coding, \nvision, medicine, law, psychology and more, without needing any special prompting. Moreover, in \nall of these tasks, GPT-4’s performance is strikingly close to human-level performance, and often \nvastly surpasses prior models such as [GPT-3.5 based] ChatGPT.”    \n88. \nThey compared the performance of GPT-4 with the performance of GPT-3 based \nsystems and found that “there is no comparis

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [34]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [35]:
print(results)

{'faithfulness': 1.0000, 'answer_relevancy': 0.9394, 'context_recall': 1.0000, 'context_precision': 0.8356, 'answer_correctness': 0.7924}


In [36]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some of the capabilities and achievem...,GPT-4 can solve novel and difficult tasks span...,[titled “Sparks of Artificial General Intellig...,GPT-4 is capable of solving novel and difficul...,1.0,0.886605,1.0,1.000000,0.849124
1,What is the purpose of the detailed paper rele...,The purpose of the detailed paper released by ...,"[challenging.” At the time, OpenAI stated that...",The purpose of the detailed paper released by ...,NaN,0.993509,1.0,1.000000,0.749245
2,How does the performance of the GPT-4 model co...,The performance of the GPT-4 model is striking...,[titled “Sparks of Artificial General Intellig...,The performance of the GPT-4 model is vastly s...,1.0,0.988203,1.0,1.000000,0.738640
3,What is Altman's role in the tech industry and...,Altman's role in the tech industry is describe...,[the Board’s ability to control Mr. Altman’s u...,Altman has been one of the dedicated high prie...,1.0,0.938596,1.0,1.000000,0.742193
4,What model did OpenAI release in 2019 that had...,GPT-2,"[challenging.” At the time, OpenAI stated that...",OpenAI released GPT-2 in 2019 with a detailed ...,1.0,0.891699,1.0,0.833333,0.961661
5,Why did OpenAI release the full version of GPT...,OpenAI released the full version of GPT-2 with...,[which needed to be trained on a specific task...,OpenAI released the full version of GPT-2 with...,1.0,0.953952,1.0,0.916667,1.000000
6,"""What is the role of GPT in deep learning and ...",GPT plays a significant role in deep learning ...,[those connections to the target language. \n7...,GPT is a large-language model developed by Ope...,NaN,0.930160,1.0,1.000000,0.737585
7,What did OpenAI's release of the GPT-2 model i...,OpenAI's release of the GPT-2 model implied th...,[which needed to be trained on a specific task...,Their publication proved to be useful to the d...,1.0,0.973487,1.0,0.638889,0.703466
8,"""What advantages does the OpenAI business mode...",The OpenAI business model offers the advantage...,[not be the first Court to hold otherwise. \n1...,The OpenAI business model offers the advantage...,1.0,0.922476,1.0,0.000000,0.483728
9,What are the defendants' fiduciary duties unde...,"Defendants owe fiduciary duties to Plaintiff, ...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"Defendants owe fiduciary duties to Plaintiff, ...",NaN,0.893286,1.0,0.833333,1.000000


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [37]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [39]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [40]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [41]:
print(response["answer"])

The plaintiff is Elon Musk.


In [42]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [43]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices under California Business and Professions Code sections 17200 et seq, and demanding an accounting, restitution, disgorgement of monies received by Defendants, prejudgment interest, an injunction against future activities, and specific performance. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [44]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [45]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [46]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [47]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some of the capabilities and achievem...,The GPT-4 model has demonstrated the ability t...,[titled “Sparks of Artificial General Intellig...,GPT-4 is capable of solving novel and difficul...,1.000000,0.950815,1.0,1.000000,0.670746
1,What is the purpose of the detailed paper rele...,The purpose of the detailed paper released by ...,[which needed to be trained on a specific task...,The purpose of the detailed paper released by ...,NaN,0.993509,1.0,1.000000,0.607843
2,How does the performance of the GPT-4 model co...,The performance of the GPT-4 model is noted to...,[titled “Sparks of Artificial General Intellig...,The performance of the GPT-4 model is vastly s...,1.000000,0.982854,0.8,0.866667,0.530487
3,What is Altman's role in the tech industry and...,Mr. Altman is heavily involved in the tech ind...,[the Board’s ability to control Mr. Altman’s u...,Altman has been one of the dedicated high prie...,1.000000,0.913963,1.0,1.000000,0.536135
4,What model did OpenAI release in 2019 that had...,OpenAI released the full version of GPT-2 in 2...,"[challenging.” At the time, OpenAI stated that...",OpenAI released GPT-2 in 2019 with a detailed ...,1.000000,0.850621,1.0,0.833333,0.805762
5,Why did OpenAI release the full version of GPT...,OpenAI released the full version of GPT-2 desp...,[which needed to be trained on a specific task...,OpenAI released the full version of GPT-2 with...,1.000000,0.953952,1.0,0.887500,0.614497
6,"""What is the role of GPT in deep learning and ...",GPT (Generative Pre-Trained Transformer) plays...,[those connections to the target language. \n7...,GPT is a large-language model developed by Ope...,1.000000,0.924462,1.0,1.000000,0.836442
7,What did OpenAI's release of the GPT-2 model i...,OpenAI's release of the GPT-2 model implied th...,[which needed to be trained on a specific task...,Their publication proved to be useful to the d...,1.000000,0.945053,1.0,0.679167,0.760741
8,"""What advantages does the OpenAI business mode...",The OpenAI business model offers the advantage...,[not be the first Court to hold otherwise. \n1...,The OpenAI business model offers the advantage...,NaN,0.926404,1.0,0.000000,0.666491
9,What are the defendants' fiduciary duties unde...,"Under California law, the defendants owe fiduc...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"Defendants owe fiduciary duties to Plaintiff, ...",0.000000,0.937159,1.0,0.833333,0.739121


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [48]:
results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9394, 'context_recall': 1.0000, 'context_precision': 0.8356, 'answer_correctness': 0.7924}

And see how our advanced retrieval modified our chain!

In [49]:
advanced_retrieval_results

{'faithfulness': 0.8857, 'answer_relevancy': 0.9302, 'context_recall': 0.9833, 'context_precision': 0.8167, 'answer_correctness': 0.7010}

In [50]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,1.000000,0.885714,-0.114286
1,answer_relevancy,0.939429,0.930163,-0.009266
2,context_recall,1.000000,0.983333,-0.016667
3,context_precision,0.835648,0.816667,-0.018981
4,answer_correctness,0.792383,0.701030,-0.091353


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [51]:
#Let us repeat the process by using "text-embedding-3-small" instead of "text-embedding-ada-002" as our embedding model:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [52]:
#Again, we store our documents alongside their new embeddings in an index:
new_vector_store = FAISS.from_documents(documents, new_embeddings)

In [53]:
#And expose our vector_store as a retriever:
new_retriever = new_vector_store.as_retriever()

In [54]:
# As we did before with "text-embedding-ada-002", we will also consider the effect of an advanced retriever:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [55]:
## As we did before with "text-embedding-ada-002", we will also consider the effect of document stuffing:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [56]:
#Now for each synthetically generated question we will retrieve contexts and proceed with generating answers with our RAG system. 

answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [57]:
# and wrap that to use with the RAGAS library:

new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [58]:
# The built-in evaluate function from RAGAS will take care of our evaluation:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [59]:
new_advanced_retrieval_results

{'faithfulness': 0.8750, 'answer_relevancy': 0.9324, 'context_recall': 0.9583, 'context_precision': 0.7720, 'answer_correctness': 0.6440}

In [60]:
#Now we will transform our results in data frames for better visualisation and comparison:

import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,1.000000,0.885714,-0.114286
1,answer_relevancy,0.939429,0.930163,-0.009266
2,context_recall,1.000000,0.983333,-0.016667
3,context_precision,0.835648,0.816667,-0.018981
4,answer_correctness,0.792383,0.701030,-0.091353


In [61]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,1.000000,0.885714,0.875000,-0.010714,-0.125000
1,answer_relevancy,0.939429,0.930163,0.932379,0.002216,-0.007050
2,context_recall,1.000000,0.983333,0.958333,-0.025000,-0.041667
3,context_precision,0.835648,0.816667,0.771991,-0.044676,-0.063657
4,answer_correctness,0.792383,0.701030,0.643959,-0.057071,-0.148424


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

**Answer** Based on the results we obtained, `text-embedding-3-small` does not appear to be significantly better than `ada`.

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [62]:
#The quick way to do that would be realising that the 'multi_context' questions in our dataset correspond to rows 6 to 11, by inspecting test_df. 
# However, a rigorous solution would be methodically filtering the rows of test_df that have ['evolution_type']== 'multi_context':
multi_test_df = test_df[test_df['evolution_type']== 'multi_context'] 

#Then proceeding with all the steps with this filtered data frame:
multi_test_questions = multi_test_df["question"].values.tolist()
multi_test_groundtruths = multi_test_df["ground_truth"].values.tolist()

answers = []
contexts = []

#We invoke our RAG pipeline to get answers and contexts:
for question in multi_test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

multi_response_dataset = Dataset.from_dict({
    "question" : multi_test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : multi_test_groundtruths
})
#Finally we get results for the metrics when only multi-context questions were used in the evaluation:
multi_results = evaluate(multi_response_dataset, metrics)

multi_results
                                        
                                        



Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

{'faithfulness': 0.9333, 'answer_relevancy': 0.9353, 'context_recall': 1.0000, 'context_precision': 0.7130, 'answer_correctness': 0.7135}

In [63]:
#We do the same procedure for the modified RAG with advanced retrieval and document stuffing: 

answers = []
contexts = []

for question in multi_test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

multi_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : multi_test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : multi_test_groundtruths
})

multi_advanced_retrieval_results = evaluate(multi_response_dataset_advanced_retrieval, metrics)
multi_advanced_retrieval_results

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

{'faithfulness': 0.9500, 'answer_relevancy': 0.9406, 'context_recall': 1.0000, 'context_precision': 0.7130, 'answer_correctness': 0.6860}

In [64]:
#Finally we repeat the procedure considering the change of embeddings model from ADA to 3-small:

answers = []
contexts = []

for question in multi_test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])



In [65]:

multi_new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : multi_test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : multi_test_groundtruths
})

multi_new_advanced_retrieval_results = evaluate(multi_new_response_dataset_advanced_retrieval, metrics)
multi_new_advanced_retrieval_results

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

{'faithfulness': 1.0000, 'answer_relevancy': 0.9155, 'context_recall': 0.9167, 'context_precision': 0.6454, 'answer_correctness': 0.7165}

In [66]:
#Now all that is left is to build a table for comparison:

df_baseline = pd.DataFrame(list(multi_results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(multi_advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(multi_new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.933333,0.950000,1.000000,0.050000,0.066667
1,answer_relevancy,0.935285,0.940566,0.915483,-0.025083,-0.019802
2,context_recall,1.000000,1.000000,0.916667,-0.083333,-0.083333
3,context_precision,0.712963,0.712963,0.645370,-0.067593,-0.067593
4,answer_correctness,0.713508,0.686036,0.716501,0.030465,0.002993


So we got similar results by using only multi-context questions for evaluation as opposed to using simple, reasoning and multi-context questions (i.e. both embedding models seem to be roughly equivalent in terms for the metrics used), at least for this specific use case